# 3802ICT Assignment 1

<p style="text-align:right">Junghee Yang (Kat) <br>
Regnier Avice<br>
Yunpeng Huang (Steven)</p>

## Project title:

## A brief description of the problem/question
<p>Amsterdam is the capital city of Netherland with a population of 870,000 with 17 million tourists every year. Many tourists these days are staying at Airbnb hosted by homeowners. This dataset is from insiderairbnb.com which contains descriptions of Airbnb listings on 6th of December 2018 in Amsterdam. The dataset includes the name of the listing, number of reviews, locations, etc. </p>

<p>This projects are consist of these problems/questions:</p>
 

<ul>
    <li><b>Classification and label prediction -</b> Can prices be predicted by the size of property and reviews?</li>
    <li><b>Association and Correlation - </b></li>
    <li><b>Clustering </b> - Do patterns in listening attributes correspond with the geographical location of the listenings?</li>
</ul>



## Dataset description
<p>The dataset contains Airbnb listings in Amsterdam on 6th December 2018 and it was retrieved from Kaggle which was downloaded from insideairbnb.com. It contains details of Airbnb listings such as prices, reviews, size of the place, etc. with 96 attributes with 20,030 data samples. </p>

## Algorithms/techniques for different learning tasks

### - Classification and label prediction
<b>Binary Classification using K-Nearest Neighbours</b>: Binary classification is one of the  Supervised Learning methods. It is classifying the elements to one or the other by measuring a series of attributes. 

K-nearest neighbours (KNN) algorithm is a type of supervised ML algorithm which can be used for both classifications as well as predictive regression problems. K-nearest neighbours (KNN) algorithm uses ‘feature similarity’ to predict the values of new datapoints which further means that the new data point will be assigned a value based on how closely it matches the points in the training set. <p style="font-size:60%;">KNN Algorithm - Finding Nearest Neighbors - Tutorialspoint. https://www.tutorialspoint.com/machine_learning_with_python/machine_learning_with_python_knn_algorithm_finding_nearest_neighbors.htm<p>

### Association and Correlation
<b>Alg:</b> Description

### Clustering
<b>K-Means (Lloyd’s Algorithm):</b> K-Means clustering is based on the idea to generate k clusters and assign each point to the cluster with the nearest mean-point. The optimal approach is NP-Hard with exponential time complexity. The Lloyd’s Algorithm is a heuristic approach to approximate the optimal result which in practice should run in linear time complexity <font color="red">reference</font>.

## Measurements to evaluate the results

### Regression
#### Mean squared error

Mean squared error is the average squared differene between the estiation and the reuslts. It is always positive value and the closer to zero, more accurate the estimation. MSE is computed as

$
{\large
\begin{align}
MSE = \frac{1}{n}\sum (Y_i - \hat{Y}_i)^2
\end{align}
}%
$

, where $
{
\begin{align}
Y
\end{align}
}%
$ is the tector of observed values of the variable being predicted, with $
{
\begin{align}
\hat{Y}
\end{align}
}%
$ being the predicted value. 

### Association and Correlation
Some metrics

### Clustering

To evaluate the effectiveness of clusters the Intracluster and Intercluster distance is measured. Effective clustering minimizes the Intracluster distance while maximizing the Intercluster distance.

For this project two popular methods are used. The Dunn Index and the Silhouette analysis.

#### Dunn Index

The goal is to maximize the value of the Dunn Index. A high Dunn Index indicates large inter cluster distance and low intra cluster distance. 

$
{\large
\begin{align}
DunnIndex = \frac{min(d(InterCluster))}{max (d(IntraCluster))} 
\end{align}
}%
$


where the inter cluster distance is the maximum distance between two points of two clusters

$
{\large
\begin{align}
d(InterCluster) = max(C_{a_i}, C_{b_j})
\end{align}
}%
$


and the intra cluster distance is maximum distance between two points of the same cluster

$
{\large
\begin{align}
d(InterCluster) = max(C_{a_i}, C_{a_j})
\end{align}
}%
$

#### Silhouette analysis

For each point the silhouette value will be caluclated which can have the range of [-1,1]. The goal is to maximize the silhouette value. Many negative values are an indication for to many or to few clusters. This value will be used to determine the right size of $k$.

$
{\large
\begin{align}
\Delta s(i) = \frac{b(i)-a(i)}{max (a(i),b(i)}
\end{align}
}%
$

where $a(i)$ is the mean distance of the point $i$ to other points of the same cluster

$
{\large
\begin{align}
\Delta a(i) = \frac{\Sigma  d(i,j)}{|C_i| -1}
\end{align}
}%
$

and $b(i)$ is the minimum (In case of more than two clusters the smallest mean is taken) mean distance of the point $i$ to other points of the other cluster. 
$
{\large
\begin{align}
\Delta b(i) = min(\frac{\Sigma  d(i,j)}{|C_k|}) \hspace{1cm}for \hspace{0.5cm} k \neq i
\end{align}
}%
$

## Libaries

In [1]:
# Import libraries and dataset 
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from scipy import stats

## Dataset

In [ ]:
# Read dataset
git_url = "https://raw.githubusercontent.com/jungheeyang/3802ICT-Assignment-1/master/listings_details.csv"
col_list = ['host_response_rate', 'host_is_superhost', 'host_identity_verified', 'zipcode', 
            'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 
            'price', 'weekly_price', 'monthly_price', 'security_deposit', 'number_of_reviews',
            'review_scores_rating', 'latitude', 'longitude', 'is_location_exact']

df = pd.read_csv(git_url, sep=',', usecols=col_list)

## Initial Data pre-processing

The initial preprocessing is conducted to convert the attribute data types into a consistent format before the data exploration is started

In [ ]:
def dollar_to_float(df):
    df = df.replace('[\$,]', '', regex=True).astype(float)
    return df


def obj_to_bool(df):
    df = df.replace('f', 'FALSE', regex=True).astype(bool)
    df = df.replace('t', 'TRUE', regex=True).astype(bool)
    return df


def percent_to_float(df):
    df = df.replace('%', '', regex=True).astype(float)
    return df


df.host_response_rate = percent_to_float(df.host_response_rate)

df.price = dollar_to_float(df.price)
df.security_deposit = dollar_to_float(df.security_deposit)
df.weekly_price = dollar_to_float(df.weekly_price)
df.monthly_price = dollar_to_float(df.monthly_price)
df.security_deposit = dollar_to_float(df.security_deposit)

df.host_is_superhost = obj_to_bool(df.host_is_superhost)
df.host_identity_verified = obj_to_bool(df.host_identity_verified)

df.head()

## Data exploration

<ul>
    <li style="font-size:16px">Number of the data samples</li>
    <ul>
        <li style="list-style-type: square">Total number of samples: 20,030</li>
    </ul>
</ul>


In [ ]:
df.count

<br>
<br>
<ul>
    <ul>
        <li style="list-style-type: square">Total number of NULL</li>
    </ul>
</ul>


In [ ]:
df.isnull().sum(axis = 0)

<br>
<br>
<ul>
    <li style="font-size:16px">Types of attributes</li>
</ul>

In [ ]:
df.dtypes

<br>
<br>
<ul>
    <li style="font-size:16px">statistical information of each attribute - Five number summery</li>
</ul>

In [ ]:
df.describe()

## Data pre-processing

In [ ]:
# New data frame for cleaned data
dfc = df.copy()

### Drop unuseful column

In [ ]:
dfc.drop(['host_response_rate'], axis=1, inplace=True)

### Remove unuseful records

In [ ]:
# Remove if beds, bedrooms, bathrooms, zipcode, review_scores_rating are missing
dfc.dropna(subset=['beds', 'bedrooms', 'bathrooms', 'zipcode', 'review_scores_rating'], inplace=True)

### Calculate missing weekly and monthly prices

In [ ]:
#Standard weekly price is 7*price
dfc['weekly_price'].fillna(dfc['price']*7, inplace=True)

#Standard monthly price is 30*price
dfc['monthly_price'].fillna(dfc['price']*30, inplace=True)

### Replace empty security deposit with 0

In [ ]:
dfc['security_deposit'].fillna(0, inplace=True)

### Exclude property with listed price of 0

In [ ]:
dfc = dfc.loc[dfc['price'] > 0]

### Remove outliers in price

In [ ]:
dfc['z_score'] = np.abs(stats.zscore(dfc['price']))
dfc = dfc[dfc.z_score < 3]

###  Check data after pre processing

In [ ]:
dfc.head()

In [ ]:
# No NULL values 
dfc.isnull().sum(axis = 0)

In [ ]:
df.count

## Visualization

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,8))

axs[0].boxplot(df['price'])
axs[0].set_title('Price data')

axs[1].boxplot(dfc['price'])
axs[1].set_title('Cleaned Price data')

fig.show

In [ ]:
dfc.to_csv('listings_details_cleaned.csv',index=False)